In [ ]:
# Loading Train Set
path = "../input/jpx-tokyo-stock-exchange-prediction/"
usecols = ["Date","SecuritiesCode","Close"]
df = pd.read_csv(f"{path}train_files/stock_prices.csv", usecols=usecols)
df = df.sort_values(["SecuritiesCode","Date"], ascending=[True,True]).ffill().bfill()
df = df[df.Date>"2021-10-20"]
df = pd.pivot_table(df,values="Close",index="Date",columns="SecuritiesCode")

In [ ]:
df

In [ ]:
df_cols = sorted(df.columns.values)
df_cols

In [ ]:
def sharpe_ratio(d, f=0.01, h=100):
    r = np.mean(d,axis=0).fillna(0)
    s = d.std()
    v = s.fillna(s.mean()).values
    v = np.where(v>f,v*h,v)
    return (r/v).fillna(0).values

In [ ]:
df_sprices = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/supplemental_files/stock_prices.csv")

In [ ]:
df_sprices

In [ ]:
udates = np.unique(df_sprices['Date'])
udates

In [ ]:
from IPython.display import display

In [ ]:
n = 15 # Best windows size in validation tests
ranks = np.array([])

for date in udates:
    prices = df_sprices[df_sprices['Date']==date].copy()
    pr = pd.pivot_table(prices,values="Close",index="Date",columns="SecuritiesCode")
    df = pd.concat([df, pr]).ffill().bfill()
    df_t = df.pct_change()       # Calculation of Returns from closings
    sr = sharpe_ratio(df_t[-n:]) # Sharpe Ratio in Rolling Window
    rank_temp = 2000-np.argsort(np.argsort(np.array(sr)))-1
    
    pr_cols = prices['SecuritiesCode'].values
    ranks_temp2 = []
    for i in range(2000):
        if df_cols[i] in pr_cols:
            ranks_temp2.append(rank_temp[i])
    ranks_temp2 = np.array(ranks_temp2)
    ranks = np.append(ranks, ranks_temp2.argsort().argsort())

In [ ]:
ranks = ranks.astype(np.int32)
ranks

In [ ]:
df_sprices['Rank'] = ranks

In [ ]:
def calc_spread_return_sharpe(df: pd.DataFrame, portfolio_size: int = 200, toprank_weight_ratio: float = 2) -> float:
    """
    Args:
        df (pd.DataFrame): predicted results
        portfolio_size (int): # of equities to buy/sell
        toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.
    Returns:
        (float): sharpe ratio
    """
    def _calc_spread_return_per_day(df, portfolio_size, toprank_weight_ratio):
        """
        Args:
            df (pd.DataFrame): predicted results
            portfolio_size (int): # of equities to buy/sell
            toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.
        Returns:
            (float): spread return
        """
        assert df['Rank'].min() == 0
        assert df['Rank'].max() == len(df['Rank']) - 1
        weights = np.linspace(start=toprank_weight_ratio, stop=1, num=portfolio_size)
        purchase = (df.sort_values(by='Rank')['Target'][:portfolio_size] * weights).sum() / weights.mean()
        short = (df.sort_values(by='Rank', ascending=False)['Target'][:portfolio_size] * weights).sum() / weights.mean()
        return purchase - short

    buf = df.groupby('Date').apply(_calc_spread_return_per_day, portfolio_size, toprank_weight_ratio)
    sharpe_ratio = buf.mean() / buf.std()
    return sharpe_ratio

calc_spread_return_sharpe(df_sprices)